# Notebook Overview

#### This notebook fine-tunes a T5-small sequence-to-sequence model to translate between Modern English and Shakespearean English using instruction-based prefixes. It loads and cleans a parallel TSV dataset, builds a bidirectional training set, and evaluates performance with ROUGE and BLEU while logging training progress and saving the best checkpoint for inference.




Installs all required libraries for fine-tuning and evaluating a T5, this includes the core transformer, evaluation, SentencePiece, and Weights & Biases integration.

In [ ]:

!pip -q install -U transformers datasets accelerate evaluate sacrebleu sentencepiece wandb rouge_score nltk


Imports core libraries for data handling, model training, and evalution.

In [ ]:
import os, re, inspect, random
import numpy as np
import torch
from datasets import load_dataset, Dataset
from transformers import (
    T5TokenizerFast,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback,
)

import evaluate
import wandb

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Make runs reproducible-ish
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if device == "cuda":
    torch.cuda.manual_seed_all(seed)


Device: cuda


This cell authenticates Weights & Biases logging using a secret stored in the environment.

In [ ]:
from google.colab import userdata
import wandb

WANDB_API_KEY = userdata.get("WANDB_API_KEY")

if WANDB_API_KEY:
    wandb.login(key=WANDB_API_KEY)
    print("W&B login ok.")
else:
    print("No WANDB_API_KEY found in Colab Secrets.")


W&B key not found in env. Paste it into WANDB_API_KEY if you want logging.


Uploads the TSV dataset into the Colab runtime.

In [ ]:
from google.colab import files

uploaded = files.upload()
print("Uploaded:", list(uploaded.keys()))


Saving data.tsv to data (2).tsv
Uploaded: ['data (2).tsv']


Loads the TSV file into a dataset and makes sure the columns are set up correctly. We also do light normalization like removing stray numbers and fixing whitespace so the model isn’t learning formatting junk instead of language.

In [ ]:
def load_tsv(path="shakes_only_data.tsv"):
    # Try no-header assumption first
    try:
        ds = load_dataset(
            "csv",
            data_files=path,
            delimiter="\t",
            column_names=["shakespeare", "modern"],
            split="train"
        )
        ex = ds[0]
        if ex.get("shakespeare") and ex.get("modern"):
            return ds
    except Exception:
        pass

    # Fallback: headered TSV
    ds = load_dataset(
        "csv",
        data_files=path,
        delimiter="\t",
        split="train"
    )
    cols = ds.column_names
    if "shakespeare" not in cols or "modern" not in cols:
        if len(cols) >= 2:
            ds = ds.rename_columns({cols[0]: "shakespeare", cols[1]: "modern"})
    return ds

number_re = re.compile(r"\b\d+\b")

def clean_text(s: str) -> str:
    if s is None:
        return ""
    s = number_re.sub("", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def clean_example(ex):
    return {
        "shakespeare": clean_text(ex.get("shakespeare", "")),
        "modern": clean_text(ex.get("modern", "")),
    }

raw = load_tsv("data.tsv").map(clean_example)

print("Columns:", raw.column_names)
print("Examples:", len(raw))
print("Sample:")
print(raw[0])


Columns: ['shakespeare', 'modern']
Examples: 38413
Sample:
{'shakespeare': 'BARNARDO Who’s there?', 'modern': 'BARNARDO Who’s there?'}


Turns each Shakespeare–Modern pair into two training examples so the model learns both directions. We add instruction-style prefixes to tell T5 which way to translate, then split everything into train and validation sets for fair evaluation.

In [ ]:
def make_bidir(ds):
    src_list, tgt_list = [], []
    for ex in ds:
        s = ex["shakespeare"].strip()
        m = ex["modern"].strip()
        if not s or not m:
            continue
        # Modern -> Shakespeare
        src_list.append(f"translate modern to shakespeare: {m}")
        tgt_list.append(s)
        # Shakespeare -> Modern
        src_list.append(f"translate shakespeare to modern: {s}")
        tgt_list.append(m)

    return Dataset.from_dict({"src": src_list, "tgt": tgt_list})

bidir = make_bidir(raw)
print("Bidir size:", len(bidir))

# 90/10 split
dataset = bidir.train_test_split(test_size=0.1, seed=seed)
dataset


Bidir size: 76810


DatasetDict({
    train: Dataset({
        features: ['src', 'tgt'],
        num_rows: 69129
    })
    test: Dataset({
        features: ['src', 'tgt'],
        num_rows: 7681
    })
})

Loads the pretrained t5-small model and its tokenizer. We define how to tokenize our instruction prompts and targets, apply truncation for efficiency,
and set up dynamic padding so batches stay clean and fast.

In [ ]:
model_name = "t5-small"
tokenizer = T5TokenizerFast.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

max_src_len = 128
max_tgt_len = 128

def preprocess(batch):
    model_inputs = tokenizer(
        batch["src"],
        max_length=max_src_len,
        truncation=True,
    )
    labels = tokenizer(
        text_target=batch["tgt"],
        max_length=max_tgt_len,
        truncation=True,
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

print(tokenized)


Map:   0%|          | 0/69129 [00:00<?, ? examples/s]

Map:   0%|          | 0/7681 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 69129
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7681
    })
})


Sets up ROUGE and BLEU so we can quantify how close the model outputs are to the references. We use these to track validation performance and compare results across models.

In [ ]:
rouge = evaluate.load("rouge")
bleu = evaluate.load("sacrebleu")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    bleu_result = bleu.compute(
        predictions=decoded_preds,
        references=[[l] for l in decoded_labels]
    )
    rouge_result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    pred_lens = [np.count_nonzero(p != tokenizer.pad_token_id) for p in preds]

    return {
        "bleu": round(bleu_result["score"], 4),
        "rouge1": round(rouge_result["rouge1"], 4),
        "rougeL": round(rouge_result["rougeL"], 4),
        "gen_len": round(float(np.mean(pred_lens)), 4),
    }


Defines the training settings like learning rate, batch size, epoch count, and checkpoint rules. Connecting with wnb here to visualize and track progress.

In [ ]:
def build_training_args():
    base_kwargs = dict(
        output_dir="t5_shakespeare_bidir",
        run_name="t5-small-shakespeare-bidir",
        learning_rate=3e-4,
        warmup_ratio=0.05,
        weight_decay=0.01,
        num_train_epochs=50,
        per_device_train_batch_size=16 if device == "cuda" else 4,
        per_device_eval_batch_size=16 if device == "cuda" else 4,
        gradient_accumulation_steps=1,
        predict_with_generate=True,

        logging_strategy="steps",
        logging_steps=50,   # frequent enough to see epoch progress
        save_strategy="epoch",
        save_total_limit=2,

        load_best_model_at_end=True,
        metric_for_best_model="rougeL",
        greater_is_better=True,

        fp16=(device == "cuda"),
        report_to="wandb",  # you asked for W&B
    )

    sig = inspect.signature(Seq2SeqTrainingArguments.__init__)
    if "evaluation_strategy" in sig.parameters:
        base_kwargs["evaluation_strategy"] = "epoch"
    else:
        base_kwargs["eval_strategy"] = "epoch"

    return Seq2SeqTrainingArguments(**base_kwargs)

training_args = build_training_args()
training_args


Seq2SeqTrainingArguments(output_dir='t5_shakespeare_bidir', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, eval_strategy=<IntervalStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=0.0003, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.05, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='t5_shakespeare_bidir/runs/Dec10_00-05-00_4ce9d949f4a3', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=50, logging_nan_inf_filter=True, save_strategy=<SaveStrategy.EPO

Builds the Seq2SeqTrainer with our model, datasets, collator, and metrics. This is sort of like a control panel where all of the training pipeline comes together.

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

print("Train size:", len(tokenized["train"]))
print("Val size:", len(tokenized["test"]))


Train size: 69129
Val size: 7681


/tmp/ipython-input-2821615650.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Runs fine-tuning for the planned number of epochs.

In [ ]:

# The trainer logs epoch automatically in its logging dict.

train_out = trainer.train()
train_out


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rougel,Gen Len
1,1.219300,1.126386,21.712500,0.612400,0.593000,18.798600
2,1.096800,1.053536,22.255200,0.619600,0.600100,18.764000
3,1.028400,1.021792,22.482300,0.621200,0.601900,18.791600
4,0.990900,0.999130,22.678000,0.623300,0.604000,18.782300
5,0.962600,0.984218,22.880400,0.625700,0.606500,18.773100
6,0.890000,0.980088,22.907700,0.626200,0.607400,18.772800
7,0.894500,0.972526,22.977700,0.627300,0.608300,18.772300
8,0.853500,0.971395,23.054200,0.628100,0.609200,18.778200
9,0.855600,0.970398,23.017500,0.628100,0.609000,18.774200
10,0.798700,0.969478,23.053400,0.628200,0.609200,18.774900


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=43210, training_loss=1.0070497670181369, metrics={'train_runtime': 4973.5516, 'train_samples_per_second': 138.993, 'train_steps_per_second': 8.688, 'total_flos': 1.7533427023970304e+16, 'train_loss': 1.0070497670181369, 'epoch': 10.0})

Runs one final evaluation on the validation set after training finishes.

In [ ]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.9713948369026184, 'eval_bleu': 23.0542, 'eval_rouge1': 0.6281, 'eval_rougeL': 0.6092, 'eval_gen_len': 18.7782, 'eval_runtime': 151.2869, 'eval_samples_per_second': 50.771, 'eval_steps_per_second': 3.179, 'epoch': 10.0}


Saves the best version of the fine-tuned T5 model and tokenizer for any future use/training

In [ ]:
trainer.save_model("t5_shakespeare_bidir_best")
tokenizer.save_pretrained("t5_shakespeare_bidir_best")

print("Saved: t5_shakespeare_bidir_best")


Saved: t5_shakespeare_bidir_best


Reloads the saved model and runs a few sample translations. This is a quick check to see how the model did. Validated by us humans.

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5_shakespeare_bidir_best").to(device)
tokenizer = T5TokenizerFast.from_pretrained("t5_shakespeare_bidir_best")

def gen(prompt, max_new_tokens=80):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=6,
        length_penalty=1.0,
        early_stopping=True,
    )
    return tokenizer.decode(out[0], skip_special_tokens=True)

tests_modern = [
    "I have a bad feeling about this.",
    "We must act quickly before night falls.",
    "The council will meet tomorrow.",
]

for t in tests_modern:
    print("Modern:", t)
    print("Shakes:", gen(f"translate modern to shakespeare: {t}"))
    print()

tests_shakes = [
    "I prithee, speak plain.",
    "Wherefore dost thou linger in the night?",
]

for t in tests_shakes:
    print("Shakes:", t)
    print("Modern:", gen(f"translate shakespeare to modern: {t}"))
    print()


Modern: I have a bad feeling about this.
Shakes: I have a bad sense of this.

Modern: We must act quickly before night falls.
Shakes: We must act quickly before night falleth.

Modern: The council will meet tomorrow.
Shakes: The council shall meet to morrow.

Shakes: I prithee, speak plain.
Modern: Please, speak plainly.

Shakes: Wherefore dost thou linger in the night?
Modern: Why do you linger in the night?



# WER Setup

In [ ]:
def translate(text):
  encoded = tokenizer(text, return_tensors="pt").to(device)
  out_tokens = model.generate(**encoded, max_length=256)
  return tokenizer.decode(out_tokens[0], skip_special_tokens=True)


In [ ]:
from google.colab import files
uploaded = files.upload()

#import file called test.tsv

Saving test.tsv to test.tsv


In [ ]:
from pathlib import Path
import re
from datasets import load_dataset

testset = load_dataset(
    "csv",
    data_files={"full": str(Path("./test.tsv"))},
    delimiter="\t",
    column_names=["shakespeare", "modern"]
)


Generating full split: 0 examples [00:00, ? examples/s]

In [ ]:
import re

def remove_numbers(row):
    row["shakespeare"] = re.sub(r"\d+", "", str(row["shakespeare"]))
    row["modern"] = re.sub(r"\d+", "", str(row["modern"]))
    return row

testset["full"] = testset["full"].map(remove_numbers)


Map:   0%|          | 0/2881 [00:00<?, ? examples/s]

In [ ]:
def too_long(testset):
    sh_words = len(str(testset["shakespeare"]).split())
    mod_words = len(str(testset["modern"]).split())
    return (sh_words <= 25) and (mod_words <= 25)

testset["full"] = testset["full"].filter(too_long)


Filter:   0%|          | 0/2881 [00:00<?, ? examples/s]

In [ ]:
inp = "<to_modern> " + str(testset["full"][1]["shakespeare"])
print("shakes: " + inp)
print("modern: " + str(testset["full"][1]["modern"]))

print(translate(inp))


shakes: <to_modern> POMPEY If the great gods be just, they shall assist The deeds of justest men.
modern: POMPEY If the great gods are just, they will help the most honest men.
POMPEY If the great gods are just, they shall assist the deeds of justest men.


In [ ]:
import csv

with open("modern_test.tsv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["original modern", "translated from early modern"])

    for i in range(len(testset["full"])):
        original = testset["full"][i]["shakespeare"]
        prompt = "<to_modern> " + str(original)

        translated = translate(prompt)
        print(i)
        print(original)
        print(translated)

        writer.writerow([str(testset["full"][i]["modern"]), translated])


Streaming output truncated to the last 5000 lines.
ANTONY I have offended reputation, A most unnoble swerving.
ANTONY I have offended my reputation, a most unnoble swerving.
518
EROS Sir, the Queen.
EROS Sir, the Queen.
519
CLEOPATRA O my lord, my lord, Forgive my fearful sails! I little thought You would have followed.
CLEOPATRA Oh my lord, my lord, I’ll beg my fearful sails! I thought you would have followed me.
520
CLEOPATRA Oh, my pardon!
CLEOPATRA Oh, my pardon!
521
CLEOPATRA Pardon, pardon!
CLEOPATRA Pardon, pardon!
522
ANTONY Fall not a tear, I say. One of them rates All that is won and lost. Give me a kiss.
ANTONY Don’t fall a tear, I say. One of them rates all that is won and lost. Give me a kiss.
523
They kiss
kiss
524
Exeunt
All exit.
525
Enter CAESAR , AGRIPPA , THIDIAS , and DOLABELLA , with others
CAESAR, AGRIPPA, THIDIAS, and DOLABELLA enter, with others.
526
CAESAR Let him appear that’s come from Antony. Know you him?
CAESAR Let him appear that’s come from Antony. Know 

In [ ]:
with open("shakespeare_test.tsv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["original early modern", "translated from modern"])

    for i in range(len(testset["full"])):
        original = testset["full"][i]["modern"]
        prompt = "<to_shakespeare> " + str(original)

        translated = translate(prompt)

        writer.writerow([str(testset["full"][i]["shakespeare"]), translated])
